In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [2]:

#participants = [ "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932",
#               "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]

participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]


#participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]



dfs = [pd.read_csv("%s%s_cleaned_ml.csv" % (cleaned_data_filepath, s)) for s in participants]

df = pd.concat(dfs, axis=0, ignore_index=True)

In [3]:
# samples are df rows mod 1200, pre is 0-199, post is 200-1199

# extract pre rows and post rows, randomly choose 200-400ms as post
pre = df[df.Time < 0]
post = df[df.Time >= 200][df.Time < 400]

/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [4]:
pre = pre.drop(columns=["Time", "E65"], axis=1)
post = post.drop(columns=["Time", "E65"], axis=1)

In [5]:
timeFrame = 200

preX = pre.values
(i,j) = preX.shape
preX = np.reshape(preX, (i // 200, j * 200))

In [6]:
postX = post.values
(i,j) = postX.shape
postX = np.reshape(postX, (i // 200, j * 200))

In [7]:
(preN, preD) = preX.shape
(postN, postD) = postX.shape

In [8]:
X = np.concatenate((preX, postX), axis=0)
(n, d) = X.shape
assert(n == preN + postN)
assert(d == postD)
assert(d == preD)

In [9]:
y = np.concatenate((np.zeros(preN), np.ones(postN)))
assert(y.shape == (n,))

In [10]:
# just use this : on next one
# from sklearn.model_selection import train_test_split 

rand_indices = np.random.permutation(n)
split = int(n * 0.8)
train_idx, valid_idx = rand_indices[:split], rand_indices[split:]
train_X, valid_X, train_y, valid_y = X[train_idx, :], X[valid_idx, :], y[train_idx], y[valid_idx]

In [11]:
from sklearn.svm import SVC
model = LinearSVC(C=1e-9, max_iter=5000)
#model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)
model.fit(train_X, train_y)

LinearSVC(C=1e-09, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=5000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:

# from sklearn.svm import SVC
# model = SVC(gamma='auto', kernel='linear')
# model.fit(train_X, train_y)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(train_X, train_y)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from dtw import DTW

# model = KNeighborsClassifier(metric=DTW)
# model.fit(train_X, train_y)

In [ ]:
Xhat = model.predict(valid_X)

In [ ]:
np.mean(Xhat != valid_y)

In [ ]:
Xhat

In [ ]:
valid_y

In [12]:
np.mean(model.predict(train_X) != train_y)

0.3020661157024793

In [ ]:
Xhat != valid_y

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(valid_y, Xhat))